In [1]:
using QuantumOptics
using LinearAlgebra
using SparseArrays

# The Boson Fock Space 

In [2]:
max_occupation_number = 2
occ_array = []
for i in 0:max_occupation_number
    push!(occ_array, i)
end

In [3]:
Nx = Ny = 2
N = Nx*Ny 
b = NLevelBasis(N)

NLevel(N=4)

In [4]:
states = bosonstates(b, [occ_array...]) #en az 0 en fazla 2 bozon
for i in 1:length(states)
    println("$i: ", states[i])
end

1: [0, 0, 0, 0]
2: [1, 0, 0, 0]
3: [0, 1, 0, 0]
4: [0, 0, 1, 0]
5: [0, 0, 0, 1]
6: [2, 0, 0, 0]
7: [1, 1, 0, 0]
8: [1, 0, 1, 0]
9: [1, 0, 0, 1]
10: [0, 2, 0, 0]
11: [0, 1, 1, 0]
12: [0, 1, 0, 1]
13: [0, 0, 2, 0]
14: [0, 0, 1, 1]
15: [0, 0, 0, 2]


In [5]:
b_mb_b = ManyBodyBasis(b, states)

ManyBody(onebodybasis=NLevel(N=4), states:15)

# The Periodic Lattice 

In [6]:
using OffsetArrays

In [7]:
lat = range(1,Nx*Ny)
latt = reshape(lat, (Nx,Ny))
lattice = OffsetArray(latt, 0:Nx-1, 0:Ny-1)
x_co = range(0, Nx-1) 
y_co = range(0, Ny-1)
arr = []
for i in 1:length(x_co)
    for j in 1:length(y_co)
       arr = [arr;x_co[i];y_co[j]]
    end
end
xy = reshape(arr, (2,Nx*Ny)) |> transpose
arr = OffsetArray(arr,0:Nx*Ny*2-1)
lattice

2×2 OffsetArray(reshape(::UnitRange{Int64}, 2, 2), 0:1, 0:1) with eltype Int64 with indices 0:1×0:1:
 1  3
 2  4

In [8]:
x = []
function PerBC()
    for i in 0:Nx-1
        for j in 0:Ny-1
            cc = [lattice[mod(j,Ny),mod(i-1,Nx)],lattice[mod(j+1,Ny),mod(i,Nx)],lattice[mod(j,Ny),mod(i+1,Nx)],lattice[mod(j-1,Ny),mod(i,Nx)]]
            push!(x,cc)
        end
    end
    return x
end

PerBC (generic function with 1 method)

# Hopping Phase Values

In [9]:
function HP(m, n, alpha)
    if abs(xy[m,1]-xy[n,1])==Nx-1
        if xy[m,1] > xy[n,1]
            A = -exp(-1im*2*pi*alpha*xy[m,2])
            #println(m," -> ",n," Kenarda x+ ve faz: ", A)
        elseif xy[m,1] < xy[n,1]
            A = -exp(1im*2*pi*alpha*xy[m,2])
            #println(m," -> ",n," Kenarda x- ve faz: ", A)
        end
    elseif m==n
        A = 0
        #println(m, " -> ", n, " Aynı noktalar ve faz: ", A)
    else
        if xy[m,1] > xy[n,1]
            A = -exp(1im*2*pi*alpha*xy[m,2])
            #println(m," -> ",n," x- ve faz: ", A)
        elseif xy[m,1] < xy[n,1]
            A = -exp(-1im*2*pi*alpha*xy[m,2])
            #println(m," -> ",n," x+ ve faz: ", A)
        else
            A = -exp(0)
            #println(m," -> ",n," x sabit ve faz: ", A)
        end
    end
end 

HP (generic function with 1 method)

# Many Body Hamiltonian

In [10]:
alpha = 1/5
S = SparseOperator(b_mb_b)
for m in 1:N
    for n in 1:N
        if m in PerBC()[n]
            S = S + HP(m, n, alpha)*transition(b_mb_b, m, n)
        end
    end
end
S
eigenenergies(dense(S))

15-element Vector{Float64}:
 -3.8042260651806123
 -2.5201470213402026
 -1.9021130325903073
 -1.2840790438404133
 -1.2360679774997896
 -0.6180339887498951
 -6.309410158126471e-16
 -3.481734714142872e-16
  0.0
  0.6180339887498951
  1.2360679774997878
  1.2840790438404117
  1.9021130325903073
  2.520147021340204
  3.8042260651806146

# Hopping Phase Values with Twisted Angle Parameters

In [11]:
Tsize=20
dx=2*pi/Tsize
dy=dx
Tx=range(start=0, stop=2*pi, step=dx) #şimdilik 1'den başlatıyorum. Chern yanlış hesaplarsam burayla ilgilenicem(!)
Ty=range(start=0, stop=2*pi, step=dy) 

0.0:0.3141592653589793:6.283185307179586

In [12]:
q=Nx
ChernAlpha=1/q
function HPTA(m, n, Tx, Ty, ChernAlpha)
    if abs(xy[m,1]-xy[n,1])==Nx-1
        if xy[m,1] > xy[n,1]
            B = -exp(-1im*2*pi*ChernAlpha*xy[m,2])*exp(-1im*Tx)
        elseif xy[m,1] < xy[n,1]
            B = -exp(1im*2*pi*ChernAlpha*xy[m,2])*exp(1im*Tx)
        end
    elseif abs(xy[m,2]-xy[n,2])==Ny-1
        if xy[m,2] > xy[n,2]
            B = -exp(-1im*Ty)
        elseif xy[m,2] < xy[n,2]
            B = -exp(1im*Ty)
        end
    elseif m==n
        B = 0
    else
        if xy[m,1] > xy[n,1]
            B = -exp(1im*2*pi*ChernAlpha*xy[m,2])
        elseif xy[m,1] < xy[n,1]
            B = -exp(-1im*2*pi*ChernAlpha*xy[m,2])
        else
            B = -exp(0)
        end
    end
end 

HPTA (generic function with 1 method)

# Many Body Chern Calculations

In [13]:
function HMTA(ChernAlpha, Tx, Ty)
    STA = SparseOperator(b_mb_b)
    for m in 1:N
        for n in 1:N
            if m in PerBC()[n]
                STA = STA + HPTA(m, n, Tx, Ty, ChernAlpha)*transition(b_mb_b, m, n)
            end
        end
    end
    return STA
end

HMTA (generic function with 1 method)

In [16]:
function HMTA(ChernAlpha, Tx, Ty)
    STA = SparseOperator(b_mb_b)
    for m in 1:N
        for n in 1:N
            if m in PerBC()[n]
                STA = STA + HPTA(m, n, Tx, Ty, ChernAlpha)*transition(b_mb_b, m, n)
            end
        end
    end
    return STA
end
#------------------------
#PROBLEM: QOJulia kütüphanesi öz-vektörleri kare matris şeklinde depolamıyor!
#------------------------
#ÇÖZÜM(Çok Yavaş):
function EigVEC(ChernAlpha, Tx, Ty)
    EigVec = spzeros(Complex{Float64},length(states),length(states))
    for m in 1:length(states)
        for n in 1:length(states)
            EigVec[m,n]=eigenstates(dense(HMTA(ChernAlpha, Tx, Ty)))[2][m].data[n]
        end
    end
    return EigVec
end
EigVEC(ChernAlpha, 2, 3)

15×15 SparseMatrixCSC{ComplexF64, Int64} with 97 stored entries:
     ⋅          ⋅                     ⋅           …       0.5-0.0im
     ⋅          ⋅                     ⋅                       ⋅    
     ⋅          ⋅                     ⋅                       ⋅    
     ⋅          ⋅            0.208073-0.454649im              ⋅    
     ⋅      0.0+0.707107im    0.07056-0.494996im              ⋅    
     ⋅          ⋅                     ⋅           …           ⋅    
 1.0+0.0im      ⋅                     ⋅                       ⋅    
     ⋅          ⋅                     ⋅                       ⋅    
     ⋅          ⋅                     ⋅                       ⋅    
     ⋅          ⋅                     ⋅              0.707107-0.0im
     ⋅          ⋅           -0.208073+0.454649im  …           ⋅    
     ⋅      0.0-0.707107im    0.07056-0.494996im              ⋅    
     ⋅          ⋅                     ⋅                  -0.5-0.0im
     ⋅          ⋅                     ⋅            

In [ ]:
ChernArray = zeros(Complex{Float64},0)
for i in range(start=1, stop=Nx*Ny, step=q) 
    j=i+q
    Sum=0
    n1=i
    n2=j
    for tx in range(start=1, stop=length(Tx))
        for ty in range(start=1, stop=length(Ty))
            w1=eigenstates(dense(HMTA(ChernAlpha, Tx[tx], Ty[ty])))[1]
            v1=EigVEC(ChernAlpha, Tx[tx], Ty[ty])
            idx1 = sortperm(w1)
            v1sorted = v1[:,idx1]
            v11 = v1sorted[:,n1:n2]
            #------------------------------------
            w2=eigenstates(dense(HMTA(ChernAlpha, Tx[tx]+dx, Ty[ty])))[1]
            v2=EigVEC(ChernAlpha, Tx[tx]+dx, Ty[ty])
            idx2 = sortperm(w2)
            v2sorted = v2[:,idx2]
            v22 = v2sorted[:,n1:n2]
            #------------------------------------
            w3=eigenstates(dense(HMTA(ChernAlpha, Tx[tx], Ty[ty]+dy)))[1]
            v3=EigVEC(ChernAlpha, Tx[tx], Ty[ty]+dy)
            idx3 = sortperm(w3)
            v3sorted = v3[:,idx3]
            v33 = v3sorted[:,n1:n2]
            #------------------------------------
            w4=eigenstates(dense(HMTA(ChernAlpha, Tx[tx]+dx, Ty[ty]+dy)))[1]
            v4=EigVEC(ChernAlpha, Tx[tx]+dx, Ty[ty]+dy)
            idx4 = sortperm(w4)
            v4sorted = v4[:,idx4]
            v44 = v4sorted[:,n1:n2]
            #----------LINK VARIABLES------------
            U1=det(adjoint(v11)*v22)
            U1=U1/abs(U1)
            U2=det(adjoint(v22)*v44)
            U2=U2/abs(U2)
            U3=det(adjoint(v33)*v44)
            U3=U3/abs(U3)
            U4=det(adjoint(v11)*v33)
            U4=U4/abs(U4)
            #----------BERRY CURVATURE-----------
            F=log(U1*U2*1/U3*1/U4)
            Sum=Sum+F
        end
    end
    Chern=1/(2*pi*1im)*Sum
    append!(ChernArray, real(Chern))
end
ChernArray

# Expected Values

In [14]:
%%capture
A = eigenstates(dense(S))

Unrecognized magic `%%capture`.

Julia does not use the IPython `%magic` syntax.   To interact with the IJulia kernel, use `IJulia.somefunction(...)`, for example.  Julia macros, string macros, and functions can be used to accomplish most of the other functionalities of IPython magics.


###### Parçacık Sayısı-Enerji Dağılımı

In [13]:
C = Array{Float64}(undef, length(states), 2)
for i in 1:length(states)
    exp = round(expect(number(b_mb_b), A[2][i])) #expected values
    C[i] = exp #expected values (first column)
    C[i,2] = A[1][i] #eigen-values (second column)
end
C
CC = sortslices(C, dims=1 ,by = x -> x[1]) #sorted according to particle numbers
show(stdout, "text/plain", CC)

15×2 Matrix{Float64}:
 0.0   0.0
 1.0  -1.84776
 1.0  -0.765367
 1.0   0.765367
 1.0   1.84776
 2.0  -3.69552
 2.0  -2.61313
 2.0  -1.53073
 2.0  -1.08239
 2.0  -8.88178e-16
 2.0  -3.40785e-17
 2.0   1.08239
 2.0   1.53073
 2.0   2.61313
 2.0   3.69552

###### Parçacık Sayısı-Açılım Katsayısı Dağılımı

In [15]:
%%capture
j = 11

Unrecognized magic `%%capture`.

Julia does not use the IPython `%magic` syntax.   To interact with the IJulia kernel, use `IJulia.somefunction(...)`, for example.  Julia macros, string macros, and functions can be used to accomplish most of the other functionalities of IPython magics.


In [16]:
A[1][j] #2 parçacıklı baz vektörü enerjisi

1.0823922002923951

In [17]:
A[2][j] #Bu baz vektöre karşılık gelen özvektörde vakum ve tek parçacıklı baz vektörleri
#açılım katsayıları sıfırdır.

Ket(dim=15)
  basis: ManyBody(onebodybasis=NLevel(N=4), states:15)
                  0.0 + 0.0im
                  0.0 + 0.0im
                  0.0 + 0.0im
                  0.0 + 0.0im
                  0.0 + 0.0im
              7.0e-17 + 0.35355339059327234im
  0.32664074121909525 - 0.13529902503654884im
  -0.3266407412190954 - 0.13529902503654873im
              3.6e-16 + 3.6e-16im
 -0.25000000000000056 - 0.24999999999999983im
             -1.0e-17 - 1.17e-15im
   0.3266407412190937 + 0.1352990250365496im
   0.2500000000000004 - 0.25000000000000006im
  0.13529902503654995 + 0.32664074121909364im
 -0.35355339059327295 - 0.0im

In [19]:
#Yukarıdaki hesaplanan özvektör içerisindeki baz vektörlerin ağırlıkları
for i in 1:length(states)    
    println(abs(A[2][j].data[i]))
end

0.0
0.0
0.0
0.0
0.0
0.35355339059327234
0.3535533905932746
0.35355339059327473
5.102800490722269e-16
0.35355339059327406
1.1658167788096346e-15
0.3535533905932735
0.35355339059327406
0.35355339059327356
0.35355339059327295
